In [1]:
import pandas as pd
# Geopandas is a geo dataframe
# import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

# Load all preprocessed data

In [2]:
all = pd.read_csv("/home/kim/data/bedmap/bedmap123_preprocessed.csv")

In [3]:
all.head()

,Unnamed: 0.1,Unnamed: 0,lon,lat,x,y,s,t,b,b_inferred,source
0,1,247,-163.00000,-78.78833,-3.572549e+05,-1.168528e+06,197.0,415.0,-218.0,True,BEDMAP1_1966-2000_AIR_BM1.csv
1,6,252,-177.10000,-78.97667,-6.077581e+04,-1.199732e+06,46.0,329.0,-283.0,True,BEDMAP1_1966-2000_AIR_BM1.csv
2,61,307,-84.81081,-77.51699,-1.355903e+06,1.231389e+05,529.0,2213.0,-1684.0,True,BEDMAP1_1966-2000_AIR_BM1.csv
3,62,308,-84.76042,-77.56345,-1.350710e+06,1.238651e+05,514.0,2213.0,-1699.0,True,BEDMAP1_1966-2000_AIR_BM1.csv
4,63,309,-84.70397,-77.61578,-1.344861e+06,1.246650e+05,505.0,2180.0,-1675.0,True,BEDMAP1_1966-2000_AIR_BM1.csv


# Subset based on Polar Stereographic projection

(y min coordinate, y max coordinate):  -1000000 , -800500
(x min coordinate, x max coordinate):  350000 , 549500

In [42]:
byrd_200km = all[(all["x"] > 350000) & (all["x"] < 549500) & (all["y"] > -1000000) & (all["y"] < -800500)]
# 470772 data points

# drop rows where no values are missing (i.e. -9999) and where bed is lower than surface
byrd_200km = byrd_200km.loc[(byrd_200km['s'] != -9999.0) & (byrd_200km['t'] != -9999.0) & (byrd_200km['b'] != -9999.0) & (byrd_200km['s'] > byrd_200km['b'])]
# down to 465760 data points

# # drop first two columns
byrd_200km = byrd_200km[byrd_200km.columns[2:]]

# Rewrite index
byrd_200km.reset_index(drop = True, inplace = True)

In [44]:
byrd_200km["b_inferred"].value_counts()

False    465129
True        631
Name: b_inferred, dtype: int64

In [45]:
byrd_200km["source"].value_counts()

NASA_2011_ICEBRIDGE_AIR_BM2.csv    315520
UTIG_2010_ICECAP_AIR_BM3.csv        97702
NASA_2017_ICEBRIDGE_AIR_BM3.csv     27877
NASA_2013_ICEBRIDGE_AIR_BM3.csv     15628
BAS_2007_AGAP_AIR_BM2.csv            8208
LDEO_2015_ROSETTA_AIR_BM3.csv         631
BEDMAP1_1966-2000_AIR_BM1.csv         193
UTIG_2004_AGASEA_AIR_BM2.csv            1
Name: source, dtype: int64

# Summary statistics on thickness, surface and bed

Thickness has the lowest std in the dataset.

In [46]:
byrd_200km["t"].describe()

count    465760.000000
mean       1583.798401
std         567.502192
min           0.000000
25%        1235.620000
50%        1627.690000
75%        1960.980000
max        3620.100000
Name: t, dtype: float64

Three rows have invalid t (thickness) values.

In [48]:
byrd_200km["s"].describe()

count    465760.00000
mean       1356.97558
std         633.30908
min        -103.78000
25%         962.92000
50%        1536.61000
75%        1871.98250
max        3478.49000
Name: s, dtype: float64

In [49]:
byrd_200km["b"].describe()

count    465760.000000
mean       -226.822839
std         660.710506
min       -3035.790000
25%        -416.410000
50%         -94.120000
75%          89.140000
max        2163.160000
Name: b, dtype: float64

# Correlation

In [53]:
(byrd_200km[["s", "t", "b"]]).corr()

,s,t,b
s,1.000000,0.398716,0.616059
t,0.398716,1.000000,-0.476747
b,0.616059,-0.476747,1.000000


# Write file

In [57]:
byrd_200km.to_csv("/home/kim/data/bedmap/byrd_200km.csv")